# Data Wrangling and Cleaning
###### by Wilson Lee
###### Data Set : China Mobile User Gemographics
###### Source Link : https://www.kaggle.com/chinapage/china-mobile-user-gemographics

In [1]:
# import libraries
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import nltk
import string
from nltk.tokenize import word_tokenize

# convert sientific notation to decimals
pd.set_option("display.float_format", lambda x:"%.2f" % x)


## Load & Merge the Data

### Load single file database

In [2]:
df_app_label = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/app_labels.csv')
df_gender_age_test = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/gender_age_test.csv')
df_gender_age_train = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/gender_age_train.csv')
df_label_categories = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/label_categories.csv')
df_phone_brand_device_model = pd.read_csv('../../Data/Raw/China Mobile User Gemographics/phone_brand_device_model.csv')
df_brand_name_translation = pd.read_csv("../../Data/Raw/brand_name_translation.csv")


### Load Multi-File database

#### Load Event List

In [3]:
wEvent_splitFolder = "../../Data/Raw/China Mobile User Gemographics/events_split"
wEvents_header_file = "events_header"
df_events_header = pd.read_csv(os.path.join(wEvent_splitFolder, wEvents_header_file))

list_df_events = []
for wRoot, wDirs, wFiles in os.walk(wEvent_splitFolder):
    for wFilename in wFiles:
        if wEvents_header_file != wFilename:
            wCurrentFilename = os.path.join(wRoot, wFilename)
            list_df_events.append(pd.read_csv(wCurrentFilename, index_col=None, header=None))
            
df_events =  pd.concat(list_df_events, axis = 0, ignore_index = True)
df_events.columns = df_events_header.columns

# de-reference loaded dataframe list
list_df_events = []

#### Load Application Events

In [4]:
wApp_Event_splitFolder = "../../Data/Raw/China Mobile User Gemographics/app_events_split"
wApp_Events_header_file = "app_events_header"
df_app_events_header = pd.read_csv(os.path.join(wApp_Event_splitFolder, wApp_Events_header_file))

list_df_app_events = [];
for wRoot, wDirs, wFiles in os.walk(wApp_Event_splitFolder):
    for wFilename in wFiles:
        if wApp_Events_header_file != wFilename:
            wCurrentFilename = os.path.join(wRoot, wFilename)
            list_df_app_events.append(pd.read_csv(wCurrentFilename, index_col=None, header=None))
            
df_app_events =  pd.concat(list_df_app_events, axis = 0, ignore_index = True)
df_app_events.columns = df_app_events_header.columns

# de-reference loaded dataframe list
list_df_app_events = []

### Preview Data

#### Gender & Age Data

In [5]:
print("Number of rows : {0}".format(len(df_gender_age_test)))
df_gender_age_test.head(3)

Number of rows : 112071


,device_id
0,1002079943728939269
1,-1547860181818787117
2,7374582448058474277


In [6]:
print("Number of rows : {0}".format(len(df_gender_age_train)))
df_gender_age_train.head(3)

Number of rows : 74645


,device_id,gender,age,group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38


#### App Label Data

In [7]:
print("Number of rows : {0}".format(len(df_app_label)))
df_app_label.head(3)

Number of rows : 459943


,app_id,label_id
0,7324884708820027918,251
1,-4494216993218550286,251
2,6058196446775239644,406


#### App Category Labels

In [8]:
print("Number of rows : {0}".format(len(df_label_categories)))
df_label_categories.head(3)

Number of rows : 930


,label_id,category
0,1,NaN
1,2,game-game type
2,3,game-Game themes


#### Device Brand and Model

In [9]:
print("Number of rows : {0}".format(len(df_phone_brand_device_model)))
df_phone_brand_device_model.head(3)

Number of rows : 187245


,device_id,phone_brand,device_model
0,-8890648629457979026,小米,红米
1,1277779817574759137,小米,MI 2
2,5137427614288105724,三星,Galaxy S4


#### Brand Name Translation

In [10]:
print("Number of rows : {0}".format(len(df_brand_name_translation)))
df_brand_name_translation.head(3)

Number of rows : 86


,chinese,english
0,三星,samsung
1,天语,Ktouch
2,海信,hisense


#### Event Data

In [11]:
print("Number of rows : {0}".format(len(df_events)))
df_events.head(10)

Number of rows : 3252950


,event_id,device_id,timestamp,longitude,latitude
0,1,29182687948017175,2016-05-01 00:55:25,121.38,31.24
1,2,-6401643145415154744,2016-05-01 00:54:12,103.65,30.97
2,3,-4833982096941402721,2016-05-01 00:08:05,106.60,29.70
3,4,-6815121365017318426,2016-05-01 00:06:40,104.27,23.28
4,5,-5373797595892518570,2016-05-01 00:07:18,115.88,28.66
5,6,1476664663289716375,2016-05-01 00:27:21,0.00,0.00
6,7,5990807147117726237,2016-05-01 00:15:13,113.73,23.00
7,8,1782450055857303792,2016-05-01 00:15:35,113.94,34.70
8,9,-2073340001552902943,2016-05-01 00:15:33,0.00,0.00
9,10,-8195816569128397698,2016-05-01 00:41:31,119.34,26.04


#### Application Event Data

In [12]:
print("Number of rows : {0}".format(len(df_app_events)))
df_app_events.head(20)

Number of rows : 32473067


,event_id,app_id,is_installed,is_active
0,2,5927333115845830913,1,1
1,2,-5720078949152207372,1,0
2,2,-1633887856876571208,1,0
3,2,-653184325010919369,1,1
4,2,8693964245073640147,1,1
5,2,4775896950989639373,1,1
6,2,-8022267440849930066,1,0
7,2,9112463267739110219,1,0
8,2,-3725672010020973973,1,0
9,2,7167114343576723123,1,1


### Clean Data

#### Drop NA & NaN

In [13]:
print("<df_app_label> Number of rows prior drop NA : {0}".format(len(df_app_label)))
df_app_label = df_app_label.dropna()
print("<df_app_label> Number of rows after drop NA : {0}".format(len(df_app_label)))
print()
print("<df_gender_age_train> Number of rows prior drop NA : {0}".format(len(df_gender_age_train)))
df_gender_age_train = df_gender_age_train.dropna()
print("<df_gender_age_train> Number of rows after drop NA : {0}".format(len(df_gender_age_train)))
print()
print("<df_label_categories> Number of rows prior drop NA : {0}".format(len(df_label_categories)))
df_label_categories = df_label_categories.dropna()
print("<df_label_categories> Number of rows after drop NA : {0}".format(len(df_label_categories)))
print()
print("<df_phone_brand_device_model> Number of rows prior drop NA : {0}".format(len(df_phone_brand_device_model)))
df_phone_brand_device_model = df_phone_brand_device_model.dropna()
print("<df_phone_brand_device_model> Number of rows after drop NA : {0}".format(len(df_phone_brand_device_model)))
print()
print("<df_brand_name_translation> Number of rows prior drop NA : {0}".format(len(df_brand_name_translation)))
df_brand_name_translation = df_brand_name_translation.dropna()
print("<df_brand_name_translation> Number of rows after drop NA : {0}".format(len(df_brand_name_translation)))
print()
print("<df_app_events> Number of rows prior drop NA : {0}".format(len(df_app_events)))
df_app_events = df_app_events.dropna()
print("<df_app_events> Number of rows after drop NA : {0}".format(len(df_app_events)))
print()
print("<df_events> Number of rows prior drop NA : {0}".format(len(df_events)))
df_events = df_events.dropna()
print("<df_events> Number of rows after drop NA : {0}".format(len(df_events)))



<df_app_label> Number of rows prior drop NA : 459943
<df_app_label> Number of rows after drop NA : 459943

<df_gender_age_train> Number of rows prior drop NA : 74645
<df_gender_age_train> Number of rows after drop NA : 74645

<df_label_categories> Number of rows prior drop NA : 930
<df_label_categories> Number of rows after drop NA : 927

<df_phone_brand_device_model> Number of rows prior drop NA : 187245
<df_phone_brand_device_model> Number of rows after drop NA : 187245

<df_brand_name_translation> Number of rows prior drop NA : 86
<df_brand_name_translation> Number of rows after drop NA : 85

<df_app_events> Number of rows prior drop NA : 32473067
<df_app_events> Number of rows after drop NA : 32473067

<df_events> Number of rows prior drop NA : 3252950
<df_events> Number of rows after drop NA : 3252950


#### Remove "unknown" Label Categories

In [14]:
print("<df_app_label> Number of rows prior <unknown> category removal : {0}".format(len(df_label_categories)))

df_label_categories = df_label_categories[(df_label_categories["category"] != "") & (df_label_categories["category"] != "NaN") & (df_label_categories["category"] != "unknown")]

print("<df_app_label> Number of rows after <unknown> category removal : {0}".format(len(df_label_categories)))


<df_app_label> Number of rows prior <unknown> category removal : 927
<df_app_label> Number of rows after <unknown> category removal : 901


#### Remove Events of Un-Installed Applications

In [15]:
print("<df_events> Number of rows prior uninstalled app removal : {0}".format(len(df_events)))

df_app_events = df_app_events[(df_app_events["is_installed"] == 1)]
print("<df_events> Number of rows after uninstalled app removal : {0}".format(len(df_events)))

df_app_events[(df_app_events["is_installed"] == 0)]


<df_events> Number of rows prior uninstalled app removal : 3252950
<df_events> Number of rows after uninstalled app removal : 3252950


,event_id,app_id,is_installed,is_active


##### Remove "is_installed" Column

In [16]:
df_app_events = df_app_events.drop("is_installed", axis=1)
df_app_events.head(3)


,event_id,app_id,is_active
0,2,5927333115845830913,1
1,2,-5720078949152207372,0
2,2,-1633887856876571208,0


#### Transform Data

##### Label Categories

###### Clean Text

In [17]:
df_trans_label_categories = df_label_categories.copy()
# to Lower case
df_trans_label_categories["category-mod"] = df_trans_label_categories["category"]
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.lower()

#dash and Spaces
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("-"," ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("_"," ")

df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("games"," game ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("game"," game ")


# play
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r'played[0-9]','play ')
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r'play[0-9]','play ')
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" playing "," play ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" played "," play ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" play "," play ")

df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("   ","  ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("  "," ")

# rpg
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" role play game "," rpg ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" rpg "," rpg game ")

# paid
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r'paid[0-9]','paid')

# low
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" lower "," low")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" low "," low ")

# higher
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" higher "," high")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" high "," high ")

# fetus
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("fetus"," pregnant baby ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("pregnancy"," pregnant baby ")

# kids
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" kids "," children ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" children "," child ")

# united states
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r"^us ","united-states ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r" us$"," united-states")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" us "," united-states ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("united states","united-states")

# united kingdom
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r"^uk ","united-kingdom ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r" uk$"," united-kingdom")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(" uk "," united-kingdom ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("united kingdom","united-kingdom")

# special words
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("jin yon","jinyon")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("xian xia","xianxia")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("flight","aviation")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("airline","aviation")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("aeronautical","aviation")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("aviations","aviation")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("shares","stock")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("puzzel","puzzle")


# replace brackets
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("("," ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(")"," ")

# replace quotes
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("'"," ")

# replace comma
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(","," ")

# replace float numbers
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r'[0-9]\.[0-9]',' ')
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r' [0-9][0-9] ',' ')

# replace period
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("."," ")

# replace single character words
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r'^. ',' ')
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r' . ',' ')
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace(r' .$',' ')

# clean spaces
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("   ","  ")
df_trans_label_categories["category-mod"] = df_trans_label_categories["category-mod"].str.replace("  "," ")


from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
nltk.download('wordnet')
nltk.download('stopwords')

# for a more complete list of stopwords and in other languages: https://www.ranks.nl/stopwords
# pritn the stop_words
stop_words = set(stopwords.words('english'))
wordnet_lemmatize = WordNetLemmatizer()
stemmer = PorterStemmer()

# custome operation
def custom_op(row):
    wWordBag = row["category-mod"].split(" ")
    # remove stop words
    wWordBag = [word for word in wWordBag if not word in stop_words]
    # remove empty strings
    wWordBag = [word for word in wWordBag if not word == ""]
    
    wNewWordBag = []
    for word in wWordBag:
        # lemmatize word
        wNewWord = wordnet_lemmatize.lemmatize(word)
        # wNewWord = stemmer.stem(wNewWord)
        wNewWordBag.append(wNewWord)

    wNewWordBag = [word for word in wNewWordBag if len(word) >= 2]

    ostring = ""
    for word in wNewWordBag:
        if not ostring == "":
            ostring += " "
        ostring += word
    return ostring

df_trans_label_categories["category-mod"] = df_trans_label_categories.apply(custom_op, axis=1)

# list = df_trans_label_categories["category-mod"].tolist()
# print(list)
# print(len(list))
df_trans_label_categories

[nltk_data] Downloading package wordnet to C:\Users\Wilson
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Wilson
[nltk_data]     Lee\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label_id,category,category-mod
1,2,game-game type,game game type
2,3,game-Game themes,game game theme
3,4,game-Art Style,game art style
4,5,game-Leisure time,game leisure time
5,6,game-Cutting things,game cutting thing
6,7,game-Finding fault,game finding fault
7,8,game-stress reliever,game stress reliever
8,9,game-pet,game pet
9,10,game-Answer,game answer
10,11,game-Fishing,game fishing


###### Create word vector

In [18]:
list_of_labels = df_trans_label_categories["category-mod"].tolist()
list_of_existing_columns = list(df_trans_label_categories)
print(list_of_existing_columns)
list_of_words = []
for label in list_of_labels:
    wordbag = label.split(" ")
    for word in wordbag:
        if word == "":
            # exclude blanks
            continue
        if word in list_of_existing_columns:
            # exclude existing columns
            continue
        if not word in list_of_words:
            list_of_words.append(word)
list_of_words.sort()
print(list_of_words)
print(len(list_of_words))

['label_id', 'category', 'category-mod']
['2g', '3d', '3g', '4g', '80', '90', 'abroad', 'academic', 'accessory', 'accommodation', 'accounting', 'action', 'active', 'activity', 'adventure', 'advertising', 'advice', 'advisory', 'aftermarket', 'aged', 'agency', 'agent', 'aggregate', 'air', 'aircraft', 'airport', 'alliance', 'amount', 'amoy', 'android', 'animation', 'answer', 'antique', 'app', 'appliance', 'application', 'area', 'around', 'arpg', 'art', 'asia', 'astrology', 'attraction', 'audiobooks', 'australia', 'austria', 'auto', 'automotive', 'aviation', 'avoid', 'baby', 'bag', 'ball', 'bank', 'banking', 'bar', 'based', 'basketball', 'bath', 'bathroom', 'beach', 'beauty', 'bed', 'behalf', 'behavior', 'belgium', 'big', 'billards', 'billiard', 'blog', 'bm', 'board', 'bobble', 'bond', 'book', 'booking', 'bookstore', 'box', 'bracelet', 'brand', 'branded', 'brazil', 'bread', 'brokerage', 'browser', 'buffet', 'bus', 'business', 'buy', 'cafe', 'calendar', 'camera', 'canada', 'capital', 'car',

###### Create "One Hot Encode" for Label Categories

In [ ]:
for word in list_of_words:
    df_trans_label_categories[word] = 0
    
for index, row in df_trans_label_categories.iterrows():
    wordbag = row["category-mod"].split(" ")
    for word in wordbag:
        if word in list_of_words:
            df_trans_label_categories.at[index, word] = 1


In [ ]:
# test transformation
df_trans_label_categories[(df_trans_label_categories["europe"]==1)]

,label_id,category,category-mod,2g,3d,3g,4g,80,90,abroad,...,woman,wonderful,word,work,world,wp,xianxia,youth,zombie,zuma
98,99,US and Europe animation,united-states europe animation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
102,103,US and Europe magic comic,united-states europe magic comic,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
842,930,"Europe, the United States and Macao (aviation)",europe united-states macao aviation,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
858,946,"Europe, the United States and Macao (Travel)",europe united-states macao travel,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
909,1001,Europe and Fantasy,europe fantasy,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Aggregate App Labels

##### Merge Label category with Application

In [ ]:
df_agg_app_label = df_app_label.copy()
df_agg_app_label = df_agg_app_label.merge(df_trans_label_categories, on="label_id", how="left")

##### Drop useless columns

In [ ]:
df_agg_app_label = df_agg_app_label.drop("label_id", axis=1)
df_agg_app_label = df_agg_app_label.drop("category", axis=1)
df_agg_app_label = df_agg_app_label.drop("category-mod", axis=1)

##### Aggregate Application word vector

In [ ]:
df_agg_app_label = df_agg_app_label.groupby(by='app_id').max()

##### Normalize word vector

In [ ]:
#for word in list_of_words:
#    df_agg_app_label[word] = df_agg_app_label[word].apply(lambda x : 1 if x > 0.5 else 0)

NewWordList = []
for word in list_of_words:
    ValueMax = df_agg_app_label[word].max()
    if ValueMax < 0.5:
        # There is no app with that attribute, we could remove it from the list to save space
        df_agg_app_label = df_agg_app_label.drop(word, axis=1)
        df_trans_label_categories = df_trans_label_categories.drop(word, axis=1)
    else:
        NewWordList.append(word)

print("Old Vector Size {0}".format(len(list_of_words)))

list_of_words = NewWordList

print("New Vector Size {0}".format(len(list_of_words)))
print(list_of_words)

Old Vector Size 729
New Vector Size 454
['3d', '80', '90', 'abroad', 'academic', 'accommodation', 'accounting', 'action', 'activity', 'adventure', 'advice', 'advisory', 'aggregate', 'air', 'aircraft', 'airport', 'alliance', 'amoy', 'animation', 'answer', 'antique', 'app', 'appliance', 'application', 'area', 'around', 'arpg', 'art', 'asia', 'astrology', 'audiobooks', 'automotive', 'aviation', 'avoid', 'baby', 'ball', 'bank', 'banking', 'based', 'basketball', 'beach', 'beauty', 'behalf', 'big', 'billards', 'blog', 'bobble', 'bond', 'book', 'booking', 'box', 'brokerage', 'browser', 'bus', 'business', 'buy', 'calendar', 'car', 'card', 'care', 'cartoon', 'casual', 'chain', 'channel', 'checkpoint', 'chess', 'child', 'chinese', 'church', 'class', 'classical', 'clock', 'coach', 'collection', 'college', 'comfortable', 'comic', 'commodity', 'community', 'comparing', 'competitive', 'complex', 'condition', 'consumer', 'contact', 'content', 'convenience', 'cool', 'cosplay', 'cost', 'coupon', 'cozy'

In [ ]:
df_agg_app_label[(df_agg_app_label["xianxia"]==1)]

,3d,80,90,abroad,academic,accommodation,accounting,action,activity,adventure,...,weibo,weight,west,western,wifi,word,world,xianxia,zombie,zuma
app_id,,,,,,,,,,,,,,,,,,,,,
2082160393290296417,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
7406605706859482641,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
9112463382285835733,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00


#### Save Data

In [ ]:
df_trans_label_categories.to_csv('../../Data/Processed/label_categories.csv', index=False)

#### Translate Phone Brand Names

In [ ]:
dict_brand_names = pd.Series(df_brand_name_translation["english"].values,index=df_brand_name_translation["chinese"]).to_dict()

print(dict_brand_names)

{'三星': 'samsung', '天语': 'Ktouch', '海信': 'hisense', '联想': 'lenovo', '欧比': 'obi', '爱派尔': 'ipair', '努比亚': 'nubia', '优米': 'youmi', '朵唯': 'dowe', '黑米': 'heymi', '锤子': 'hammer', '酷比魔方': 'koobee', '美图': 'meitu', '尼比鲁': 'nibilu', '一加': 'oneplus', '优购': 'yougo', '诺基亚': 'nokia', '糖葫芦': 'candy', '中国移动': 'ccmc', '语信': 'yuxin', '基伍': 'kiwu', '青橙': 'greeno', '华硕': 'asus', '夏新': 'panosonic', '维图': 'weitu', '艾优尼': 'aiyouni', '摩托罗拉': 'moto', '乡米': 'xiangmi', '米奇': 'micky', '大可乐': 'bigcola', '沃普丰': 'wpf', '神舟': 'hasse', '摩乐': 'mole', '飞秒': 'fs', '米歌': 'mige', '富可视': 'fks', '德赛': 'desci', '梦米': 'mengmi', '乐视': 'lshi', '小杨树': 'smallt', '纽曼': 'newman', '邦华': 'banghua', 'E派': 'epai', '易派': 'epai', '普耐尔': 'pner', '欧新': 'ouxin', '西米': 'ximi', '海尔': 'haier', '波导': 'bodao', '糯米': 'nuomi', '唯米': 'weimi', '酷珀': 'kupo', '谷歌': 'google', '昂达': 'ada', '聆韵': 'lingyun', '小米': 'xiaomi', '酷派': 'coolpad', '华为': 'huawei', '金立': 'gionee', '中兴': 'zte', '魅族': 'meizu', '康佳': 'Kanga', '奇酷': 'Qi Cool', '欧博信': 'Ou Bo Xin', '金星数码'

In [ ]:
df_phone_brand_device_model["phone_brand"] = df_phone_brand_device_model["phone_brand"].apply( lambda x : x if not x in dict_brand_names else dict_brand_names[x])
df_phone_brand_device_model["phone_brand"].unique()

array(['xiaomi', 'samsung', 'SUGAR', 'huawei', 'vivo', 'meizu', 'coolpad',
       'Ktouch', 'OPPO', 'zte', 'gionee', 'lenovo', 'hisense', '索尼', 'LG',
       'HTC', 'Cool ratio', 'Kanga', 'Qi Cool', 'Ou Bo Xin', 'obi', 'TCL',
       'ipair', 'nubia', 'youmi', 'LOGO', 'dowe', 'heymi', 'hammer',
       'koobee', 'meitu', 'nibilu', 'oneplus', 'yougo', 'nokia', 'candy',
       'ccmc', 'yuxin', 'kiwu', 'greeno', 'asus', 'panosonic', 'weitu',
       'aiyouni', 'moto', 'xiangmi', 'micky', 'bigcola', 'wpf', 'hasse',
       'mole', 'fs', 'mige', 'fks', 'desci', 'mengmi', 'lshi', 'smallt',
       'newman', 'banghua', 'epai', 'pner', 'ouxin', 'ximi', 'haier',
       'bodao', 'nuomi', 'weimi', 'kupo', 'google', 'ZUK', 'Yitong',
       'ada', 'lingyun', 'jxd', '广信', '至尊宝', 'lephone', 'noain', '欧奇',
       '贝尔丰', 'MIL', '斐讯', '优语', 'Lovme', '白米', '百加', '宝捷讯', '果米', '首云',
       '瑞米', '瑞高', '台电', '丰米', '唯比', '长虹', '大Q', '鲜米', 'Pioneer', '恒宇丰',
       '虾米', '凯利通', '青葱', '奥克斯', '蓝魔', '智镁', 'philips', 's

#### Merge Phone Model and User Info

##### Users with Age Group

In [ ]:
df_agg_user_info_w_age = df_gender_age_train.copy()
df_agg_user_info_w_age = df_agg_user_info_w_age.merge(df_phone_brand_device_model, on="device_id", how="left")
df_agg_user_info_w_age = df_agg_user_info_w_age.dropna()
df_agg_user_info_w_age.head(5)

,device_id,gender,age,group,phone_brand,device_model
0,-8076087639492063270,M,35,M32-38,xiaomi,MI 2
1,-2897161552818060146,M,35,M32-38,xiaomi,MI 2
2,-8260683887967679142,M,35,M32-38,xiaomi,MI 2
3,-4938849341048082022,M,30,M29-31,xiaomi,红米note
4,245133531816851882,M,30,M29-31,xiaomi,MI 3


##### Users Without Age Group

In [ ]:
df_agg_user_info_w_out_age = df_gender_age_test.copy()
df_agg_user_info_w_out_age = df_agg_user_info_w_out_age.merge(df_phone_brand_device_model, on="device_id", how="left")
df_agg_user_info_w_out_age = df_agg_user_info_w_out_age.dropna()
df_agg_user_info_w_out_age.head(5)

,device_id,phone_brand,device_model
0,1002079943728939269,xiaomi,小米note
1,-1547860181818787117,xiaomi,红米2
2,7374582448058474277,huawei,Y523-L176
3,-6220210354783429585,huawei,荣耀6
4,-5893464122623104785,xiaomi,MI 2


#### Transform Events database

##### Extract Events for Users with Age Info

For our research we are only intrested in users with are information. Will discard the data for users without age information.

In [ ]:
df_events_users_w_age = df_agg_user_info_w_age.merge(df_events, on="device_id",how="inner")
df_events_users_w_age

,device_id,gender,age,group,phone_brand,device_model,event_id,timestamp,longitude,latitude
0,-8260683887967679142,M,35,M32-38,xiaomi,MI 2,2479656,2016-05-01 14:23:37,0.00,0.00
1,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,280934,2016-05-04 14:10:03,119.57,31.75
2,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,363585,2016-05-04 14:13:30,119.57,31.75
3,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,935225,2016-05-06 18:51:15,0.00,0.00
4,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,1551108,2016-05-04 14:09:29,119.57,31.75
5,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,1901681,2016-05-04 14:10:00,119.57,31.75
6,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,1969680,2016-05-04 14:08:17,119.59,31.75
7,7477216237379271436,F,37,F33-42,huawei,荣耀6 plus,2616450,2016-05-04 14:08:23,119.57,31.75
8,6352067998666467520,M,32,M32-38,huawei,荣耀畅玩4X,7665,2016-05-05 09:36:50,0.00,0.00
9,6352067998666467520,M,32,M32-38,huawei,荣耀畅玩4X,223251,2016-05-05 10:44:38,0.00,0.00


##### Calulate Average Locations

In [ ]:
# Get a list of valid locations
df_valid_eventLocation = df_events_users_w_age.copy()
df_valid_eventLocation = df_valid_eventLocation[(df_valid_eventLocation["longitude"] > -179.99) & ((df_valid_eventLocation["latitude"] < -0.001)|(df_valid_eventLocation["latitude"] > 0.001))]
df_valid_eventLocation = df_valid_eventLocation[(df_valid_eventLocation["longitude"] < 179.99) & ((df_valid_eventLocation["latitude"] < -0.001)|(df_valid_eventLocation["latitude"] > 0.001))]
df_valid_eventLocation = df_valid_eventLocation[((df_valid_eventLocation["longitude"] < -0.001)|(df_valid_eventLocation["longitude"] > 0.001)) & ((df_valid_eventLocation["latitude"] < -0.001)|(df_valid_eventLocation["latitude"] > 0.001))]

# Get Average Location of the user using valid results
df_valid_eventLocation = df_valid_eventLocation.groupby("device_id").mean();
Available_Devices = df_valid_eventLocation.index

mean_longitude = df_valid_eventLocation["longitude"].mean();
mean_latitude = df_valid_eventLocation["latitude"].mean();

print("Average Location of users : {0} deg longitude, {1} deg latitude".format(mean_longitude, mean_latitude))


Average Location of users : 110.01291235253137 deg longitude, 30.494667495690518 deg latitude


##### Fix Location Data

In [ ]:
print("Fix Location Start")

df_trans_event = df_events_users_w_age.copy()
    
for index, row in df_trans_event.iterrows():
    location = [row["longitude"], row["latitude"]]
    device_id = row["device_id"]
    if location[1] < 0.001:
        if location[1] > -0.001:
            isbad = False
            if location[0] < -179.99:
                isbad = True
            elif location[0] > 179.99:
                isbad = True
            elif location[0] < 0.001:
                if location[0] > -0.001:
                    isbad = True
            if True == isbad:
                if not device_id in Available_Devices:
                    location = [mean_longitude, mean_latitude]
                else:
                    location = [df_valid_eventLocation["longitude"][device_id], df_valid_eventLocation["latitude"][device_id]]
    df_trans_event.at[index, "longitude"] = location[0]
    df_trans_event.at[index, "latitude"] = location[1]
            
print("Fix Location End")

Fix Location Start
Fix Location End


##### Set Encoding for Event time

In [ ]:
df_trans_event_dt = df_trans_event.copy()
df_trans_event_dt['timestamp'] =pd.to_datetime(df_trans_event_dt['timestamp'])
df_trans_event_dt["hour"] = 0
df_trans_event_dt["hour"] = df_trans_event_dt['timestamp'].dt.hour
df_trans_event_dt["day_of_week"] = 0
df_trans_event_dt["day_of_week"] = df_trans_event_dt["timestamp"].dt.dayofweek

df_trans_event_dt = df_trans_event_dt.drop("timestamp", axis=1)
df_trans_event_dt.tail(5)


,device_id,gender,age,group,phone_brand,device_model,event_id,longitude,latitude,hour,day_of_week
1220469,89181010588227347,M,25,M23-26,xiaomi,MI 2S,2218742,117.08,26.61,19,3
1220470,89181010588227347,M,25,M23-26,xiaomi,MI 2S,2243803,117.08,26.61,17,4
1220471,89181010588227347,M,25,M23-26,xiaomi,MI 2S,2336186,119.26,26.05,14,2
1220472,89181010588227347,M,25,M23-26,xiaomi,MI 2S,3139128,119.26,26.05,14,2
1220473,89181010588227347,M,25,M23-26,xiaomi,MI 2S,3176234,117.08,26.61,17,4


#### Aggregate Application Event 

##### Calculate Number of Events per File

In [ ]:
last_event_id = df_app_events["event_id"].max()
number_of_files = 50;
number_of_events_per_file = np.ceil(last_event_id/number_of_files)

print("There are {0} number of events.".format(last_event_id))
print("They will be split into {0} files. {1} events per file.".format(number_of_files, number_of_events_per_file))


There are 3252948 number of events.
They will be split into 50 files. 65059.0 events per file.


##### Merging App Label Category and User Info into Application Event and Saving Data in Multiple Files

In [ ]:
print("Function start ...")

df_agg_user_info_w_age = df_trans_event_dt

for i in range(0, number_of_files):
    min_event_id = i*number_of_events_per_file;
    max_event_id = (i+1)*number_of_events_per_file;
    
    # get event slice
    df_app_event_slice = df_app_events[(df_app_events["event_id"] >= min_event_id) & (df_app_events["event_id"] < max_event_id)]
    
    # merge word vector into event label
    df_app_event_slice = df_app_event_slice.merge(df_agg_app_label, on="app_id", how="left")
    
    # drop App_ID
    df_app_event_slice = df_app_event_slice.drop("app_id", axis=1)
    
    # remove unknown apps
    df_app_event_slice = df_app_event_slice.dropna()
    
    # seperate into active and inactive apps
    df_app_event_slice_active = df_app_event_slice[(df_app_event_slice["is_active"] == 1 )]
    
    # aggregate event word vector
    df_app_event_slice_active = df_app_event_slice_active.groupby("event_id").sum()
    
    # get the events of the specified range to reduce computations in merge
    df_agg_user_info_w_age_slice =  df_agg_user_info_w_age[(df_trans_event_dt["event_id"] >= min_event_id) & (df_agg_user_info_w_age["event_id"] < max_event_id)]
    
    # Merge data sets
    df_app_event_slice_active_w_age = df_app_event_slice_active.merge(df_agg_user_info_w_age_slice, on="event_id", how="inner")
  
    # Set File name
    filename_active_w_age = '../../Data/Processed/users_with_age/active_apps/app_active_age_events_{:04d}.csv'.format(i+1)
    
    # Save data to csv
    df_app_event_slice_active_w_age.to_csv(filename_active_w_age, index=False)
    
print("Function Complete")

Function start ...


### Get List of Phone Model and brand

#### Count Number of Devices with User Age available

In [ ]:
df_phone_list = df_agg_user_info_w_age[["phone_brand","device_model"]].copy()
df_phone_list["count"] = 1
df_phone_list.head(4)

In [ ]:
df_phone_list = df_phone_list.groupby(by=['phone_brand', 'device_model'])['count'].sum()
df_phone_list

#### Save Device Model Data

In [ ]:
df_phone_list.to_csv('../../Data/Processed/device_model_count.csv', index=True, header=True)